# Первые пипы и скачивание датасета

In [1]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xjf LJSpeech-1.1.tar.bz2

--2021-12-06 10:06:53--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 174.138.79.61
Connecting to data.keithito.com (data.keithito.com)|174.138.79.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [application/octet-stream]
Saving to: ‘LJSpeech-1.1.tar.bz2’

LJSpeech-1.1.tar.bz 100%[===================>]   2.56G  36.0MB/s    in 74s     

2021-12-06 10:08:07 (35.6 MB/s) - ‘LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [2]:
!pip install librosa

In [3]:
!pip install torch==1.10.0+cu111 torchaudio==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


# Модель FastSpeech

Источник вдохновения:  https://github.com/xcmyz/FastSpeech

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import math

## Позиционное кодирование

Код для позиционного кодирования я нашла https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Однако, там была какая-то бага с тем, что если  размерность эмбеддингов нечетная, то все падает, поэтому я немного подправила

In [5]:
class PositionalEncodingBlock(nn.Module):
    def __init__(self,
                 embed_dim, #Размерность вектора
                 max_len #Максимальная длина последовательности
                 ):
        
        super(PositionalEncodingBlock, self).__init__()
        #В туториале какая-то бага, если размерность нечетная, то у меня падало
        if embed_dim % 2 != 0:
            embed_dim += 1
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-math.log(10000.0) / embed_dim))
        self.position_embeddings = torch.zeros(max_len, embed_dim, requires_grad=False)
        self.position_embeddings[:,0::2] = torch.sin(position * div_term)
        self.position_embeddings[:,1::2] = torch.cos(position * div_term)
        self.position_embeddings = self.position_embeddings.to(DEVICE)
    def forward(self, x):
      
      return x + self.position_embeddings[:x.shape[1],:x.shape[-1]]





## Multihead Attention 

Как ясно из названия, здесь реализуется многоголовое self-attention

In [6]:
import math
class MultiheadAttentionBlock(nn.Module):
    def __init__(self,
                 embed_dim, 
                 num_heads, 
                 #Это все, что есть в pytorch
                 #по-моему, обойдусь без него
                 #TODO: имплеменитровать или удалить
                 dropout=0.0,
                 bias=True, 
                 add_bias_kv=False, 
                 add_zero_attn=False, 
                 kdim=None, 
                 vdim=None, 
                 batch_first=False, 
                 device=None, 
                 dtype=None):
        super(MultiheadAttentionBlock, self).__init__()
        self.num_heads = num_heads
       
        self.embed_dim = embed_dim
        self.head_dim = embed_dim // num_heads
        #Так как после мултиголовости у нас идет склейка, то надо, чтобы размерность делилась
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"
        #Создам просто массив, который будет хранить головы
        #Каждая голова - это тупл из трех линейных слоев (для Q,K,V соответсвтенно)
        '''UPD это не обучается
        self._heads = []
        for _ in range(self.num_heads):
            self._heads.append((
                nn.Linear(self.embed_dim, self.head_dim).to(DEVICE), #Это типа Q
                nn.Linear(self.embed_dim, self.head_dim).to(DEVICE), #Это типа K
                nn.Linear(self.embed_dim, self.head_dim).to(DEVICE)  #Это типа V
            ))'''
        self.q_heads, self.k_heads, self.v_heads = [], [], []
        for _ in range(self.num_heads):
            self.q_heads.append(nn.Linear(self.embed_dim, self.head_dim).to(DEVICE))
            self.k_heads.append(nn.Linear(self.embed_dim, self.head_dim).to(DEVICE))
            self.v_heads.append(nn.Linear(self.embed_dim, self.head_dim).to(DEVICE))
        self.q_heads, self.k_heads, self.v_heads = nn.ModuleList(self.q_heads), nn.ModuleList(self.k_heads),  nn.ModuleList(self.v_heads)
        self.o = nn.Linear(self.embed_dim, self.embed_dim)
    def _attention(self, Q, K, V):
        #softmax(QK^T / d_k) V
        return torch.matmul(
            F.softmax(
                torch.matmul(
                    Q,
                    torch.transpose(
                        K, 
                        -2, 
                        -1)
                )/ math.sqrt(Q.shape[-1]),
                dim=-1),
            V
        )
    
    def forward(self, x):

        head_results = []
        for lin_Q, lin_K, lin_V in zip(self.q_heads, self.k_heads, self.v_heads):
            #print(lin_Q.device)
            head_results.append(self._attention(lin_Q(x), lin_K(x), lin_V(x)))
        res = torch.cat(head_results, -1)
        return self.o(res)
        

In [7]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## FFT блок 
Таким образом, что делает FFT блок:



*   Прогоняет через блок self-attention
*   Дропаутит
*   Суммирует с тем, что было изначально
*   Берет норму (layer)
*   Транспонирует
*   Дважды прогоняет через свертку с ReLU
*   Дропаутит 
*   Суммирует с тем, что было до сверток
*   Транспонирует
*   Берет норму (layer)



In [8]:
class FFTBlock(nn.Module):
    def __init__(self, 
               embed_dim, #Это обычная размерность (embed_dim для MHatt)
               num_heads, #Это для MHatt
               hidden_dim, #Это для conv1d (типо к-во каналов на выходе первого conv1d)
               kernel_size_1, #Это для первого блока conv1d
               kernel_size_2, #Это для второго блока conv1d
               dropout1=0.1, #Это dropout после MHatt
               dropout2=0.1 #Это dropout после conv
               ):
        super(FFTBlock, self).__init__()
        self.mha = MultiheadAttentionBlock(embed_dim, num_heads)
        self.dp1 = nn.Dropout(dropout1)
        self.ln1 = nn.LayerNorm(embed_dim)

        self.conv1 = nn.Conv1d(embed_dim, 
                             hidden_dim, 
                             kernel_size=kernel_size_1,
                             padding='same')
        self.r1 = nn.ReLU()
        self.conv2 = nn.Conv1d(hidden_dim, 
                             embed_dim, 
                             kernel_size=kernel_size_2,
                             padding='same')
        self.r2 = nn.ReLU()
        self.dp2 = nn.Dropout(dropout2)

        self.ln2 = nn.LayerNorm(embed_dim)
    def forward(self, x):
        out = x + self.dp1(self.mha(x))
        out = self.ln1(out)
        out = out.transpose(-2,-1)
        new_out = self.dp2(self.r2(self.conv2(self.r1(self.conv1(out)))))
      
        new_out = out + new_out
        new_out = new_out.transpose(-2, -1)
        new_out = self.ln2(new_out)
      
        return new_out




## Encoder

Энкодер состоит из блока эмбеддингов, блока позиционного кодирования  и N блоков FFT

In [9]:
class EncoderBlock(nn.Module):
    def __init__(self,
                 #Это для эмбеддингов
                 vocab_size,
                 #Это к-во блоков
                 n_layers,
                 #Это для FFT блоков (они же одинаковые, надеюсь)
                 embed_dim, #Это важно не только для FFT блока
                 num_heads,
                 hidden_dim, 
                 kernel_size_1, 
                 kernel_size_2, 
                 dropout1=0.1,
                 dropout2=0.1
                 ):
        super(EncoderBlock, self).__init__()
        #self.embed_dim = emded_dim
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.pe = PositionalEncodingBlock(embed_dim, vocab_size + 2)
        self.FFT_seq = nn.Sequential(
            *[
                  FFTBlock(embed_dim, 
                           num_heads,
                           hidden_dim,
                           kernel_size_1, 
                           kernel_size_2, 
                           dropout1, 
                           dropout2 
                           )   
                  
                  for _ in range(n_layers)                  
             ]
             )
    def forward(self, x):
        return self.FFT_seq(self.pe(self.emb(x)))


## Декодер

В декодере все полегче, там только позиционное кодирование N FFT блоков и финальный линейный слой

In [10]:
class DecoderBlock(nn.Module):
    def __init__(self,
                 #Это к-во блоков
                 n_layers,
                 # Это к-во выходов для финального слоя
                 out_dim,
                 #Это для позиционного кодирования
                 max_len,
                 #Это для FFT блоков (они же одинаковые, надеюсь)
                 embed_dim, #Это важно не только для FFT блока
                 num_heads,
                 hidden_dim, 
                 kernel_size_1, 
                 kernel_size_2, 
                 dropout1=0.1,
                 dropout2=0.1
                 ):
        super(DecoderBlock, self).__init__()
        self.pe = PositionalEncodingBlock(embed_dim, max_len)
        self.FFT_seq = nn.Sequential(
            *[
                  FFTBlock(embed_dim, 
                           num_heads,
                           hidden_dim,
                           kernel_size_1, 
                           kernel_size_2, 
                           dropout1, 
                           dropout2 
                           )   
                  
                  for _ in range(n_layers)                  
             ]
             )
        self.final_layer = nn.Linear(embed_dim, out_dim)
    def forward(self, x):
        #по-моему, тут нужно транспонирование
        return self.final_layer(self.FFT_seq(self.pe(x))).transpose(-2,-1)


## Duration Predictor
В этом блоке делается:


*   conv1d + layer norm + relu + dropout
*   conv1d + layer norm + relu + dropout
*   Линейный слой
*   ReLU


По-моему, вторая conv принимает hidden_dim и выдает hidden_dim, так что не нужен лишний параметр. Линейный слой возвращает 1 число


In [11]:
class DurationPredictorBlock(nn.Module):
    def __init__(self, 
                 embed_dim,
                 hidden_dim,
                 kernel_size_1,
                 kernel_size_2,
                 dropout1=0.1,
                 dropout2=0.1):
        super(DurationPredictorBlock, self).__init__()
        self.conv1 =  nn.Conv1d(embed_dim, 
                             hidden_dim, 
                             kernel_size=kernel_size_1,
                             padding='same')
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.r1 = nn.ReLU()
        self.dp1 = nn.Dropout(dropout1)

        self.conv2 = nn.Conv1d(hidden_dim, 
                             hidden_dim, 
                             kernel_size=kernel_size_2,
                             padding='same')
        self.ln2 = nn.LayerNorm(hidden_dim)
        self.r2 = nn.ReLU()
        self.dp2 = nn.Dropout(dropout2)

        self.lin = nn.Linear(hidden_dim, 1)
        self.r3 = nn.ReLU()

    def forward(self, x):
        out = self.dp1(self.r1(self.ln1(self.conv1(x.transpose(-2,-1)).transpose(-2,-1))))

        out = self.dp2(self.r2(self.ln2(self.conv2(out.transpose(-2,-1)).transpose(-2,-1))))

        out = self.r3(self.lin(out))

        return out.squeeze(-1)




## Length Regulator

В этом блоке мы используем Duration Predictor и хитрыми питоновскими штуками (циклами) корректируем длины

In [12]:
import numpy as np
class LengthRegulatorBlock(nn.Module):
    def __init__(self,
                 embed_dim,
                 hidden_dim,
                 kernel_size_1,
                 kernel_size_2,
                 dropout1=0.1,
                 dropout2=0.1
                 ):
        super(LengthRegulatorBlock, self).__init__()
        self.dur_predictor = DurationPredictorBlock(embed_dim,
                 hidden_dim,
                 kernel_size_1,
                 kernel_size_2,
                 dropout1,
                 dropout2)
    def forward(self, x, D=None, alpha=1.0):
        len_pred = self.dur_predictor(x)

        if D is None:
            D = (len_pred.squeeze(-1) + 0.5) * alpha
        out = self.LR(x, D)
        return out, len_pred
    
    #Чтобы получить H_mel надо сделать маску, на которую потом умножить H_pho
    def _create_alignment(self, D, expand_max_len):
        alignment = torch.zeros(D.shape[0], D.shape[-1], expand_max_len).to(DEVICE)
        for i in range(D.shape[0]):
            phoneme_start = 0
            for j in range(D.shape[-1]):
                phoneme_dur = round(D[i][j].item())
                
                alignment[i, j, phoneme_start:phoneme_start+phoneme_dur] = 1
                phoneme_start = phoneme_start + phoneme_dur
        return alignment

    
    def LR(self, x, D):
        expand_max_len = round(D.sum(-1).max().item())
        
        alignment = self._create_alignment(D, expand_max_len)
        alignment = alignment.to(DEVICE)
        #print('LR',x.shape, alignment.shape, D.shape)

        return x.transpose(-2,-1) @ alignment


    

## FastSpeech

Теперь нам осталось только сделать Лего из блоков

In [13]:
class FastSpeechModel(nn.Module):
    def __init__(self,
                 #Это для энкодера
                 vocab_size,
                 n_layers_enc,
                 embed_dim, 
                 num_heads_enc,
                 hidden_dim_enc, 
                 kernel_size_1_enc, 
                 kernel_size_2_enc, 
                 dropout1_enc,
                 dropout2_enc,
                 #Это для length_regulator
                 #embed_dim, Не надо, так как нужно то же самое, что и в энкодере
                 hidden_dim_lr,
                 kernel_size_1_lr,
                 kernel_size_2_lr,
                 dropout1_lr,
                 dropout2_lr,
                 #Это для декодера
                 n_layers_dec,
                 out_dim,
                 #embed_dim, Не надо, так как нужно то же самое, что и в энкодере
                 num_heads_dec,
                 hidden_dim_dec, 
                 kernel_size_1_dec, 
                 kernel_size_2_dec, 
                 dropout1_dec,
                 dropout2_dec):
        super(FastSpeechModel, self).__init__()
        self.embed_dim = embed_dim
        self.encoder = EncoderBlock(
                 vocab_size,
                 n_layers_enc,
                 embed_dim, #Это важно не только для FFT блока
                 num_heads_enc,
                 hidden_dim_enc, 
                 kernel_size_1_enc, 
                 kernel_size_2_enc, 
                 dropout1_enc,
                 dropout2_enc
                 )
        self.length_regulator = LengthRegulatorBlock(
                 embed_dim,
                 hidden_dim_lr,
                 kernel_size_1_lr,
                 kernel_size_2_lr,
                 dropout1_lr,
                 dropout2_lr
                 )
        self.decoder = DecoderBlock(
                 n_layers_dec,
                 out_dim,
                 vocab_size + 2,
                 embed_dim, 
                 num_heads_dec,
                 hidden_dim_dec, 
                 kernel_size_1_dec, 
                 kernel_size_2_dec, 
                 dropout1_dec,
                 dropout2_dec
                 )
    
    def forward(self, x, D=None):
        #print('start', x.shape)
        x = self.encoder(x)
        #print('after encoder', x.shape)
        x, len_pred = self.length_regulator(x, D)
        #print('len_pred', len_pred.shape)
        x = x.transpose(-2,-1)
        #print(x.shape)
        x = self.decoder(x).transpose(-2, -1)
        return x, len_pred

## Функция потерь

Для удобства сделаю функцию потерь отдельным блоком

In [14]:
class LossFunctionBlock(nn.Module):
    def __init__(self):
        super(LossFunctionBlock, self).__init__()

    def forward(self, mel_real, mel_pred, dur_real=None, dur_pred=None):
        #Возможно, мелспеки разной длины
        min_len = min(mel_real.shape[-1], mel_pred.shape[-1])
        mel_loss = F.mse_loss(mel_real[...,:min_len], mel_pred[...,:min_len])
        dur_loss = None
        if dur_real is not None and dur_pred is not None:
            dur_loss = F.mse_loss(dur_real, dur_pred)
        return mel_loss, dur_loss

# Featurizer

(дано)

In [15]:
from IPython import display
from dataclasses import dataclass

import torch
from torch import nn

import torchaudio

import librosa
from matplotlib import pyplot as plt


@dataclass
class MelSpectrogramConfig:
    sr: int = 22050
    win_length: int = 1024
    hop_length: int = 256
    n_fft: int = 1024
    f_min: int = 0
    f_max: int = 8000
    n_mels: int = 80
    power: float = 1.0

    # value of melspectrograms if we fed a silence into `MelSpectrogram`
    pad_value: float = -11.5129251


class MelSpectrogram(nn.Module):

    def __init__(self, config: MelSpectrogramConfig):
        super(MelSpectrogram, self).__init__()

        self.config = config

        self.mel_spectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=config.sr,
            win_length=config.win_length,
            hop_length=config.hop_length,
            n_fft=config.n_fft,
            f_min=config.f_min,
            f_max=config.f_max,
            n_mels=config.n_mels
        )

        # The is no way to set power in constructor in 0.5.0 version.
        self.mel_spectrogram.spectrogram.power = config.power

        # Default `torchaudio` mel basis uses HTK formula. In order to be compatible with WaveGlow
        # we decided to use Slaney one instead (as well as `librosa` does by default).
        mel_basis = librosa.filters.mel(
            sr=config.sr,
            n_fft=config.n_fft,
            n_mels=config.n_mels,
            fmin=config.f_min,
            fmax=config.f_max
        ).T
        self.mel_spectrogram.mel_scale.fb.copy_(torch.tensor(mel_basis))

    def forward(self, audio: torch.Tensor) -> torch.Tensor:
        """
        :param audio: Expected shape is [B, T]
        :return: Shape is [B, n_mels, T']
        """

        mel = self.mel_spectrogram(audio) \
            .clamp_(min=1e-5) \
            .log_()

        return mel

In [16]:
featurizer = MelSpectrogram(MelSpectrogramConfig())

---

# Dataset

In [17]:
class LJSpeechDataset(torchaudio.datasets.LJSPEECH):

    def __init__(self, root):
        super().__init__(root=root)
        self._tokenizer = torchaudio.pipelines.TACOTRON2_GRIFFINLIM_CHAR_LJSPEECH.get_text_processor()

    def __getitem__(self, index: int):
        waveform, _, _, transcript = super().__getitem__(index)
        waveform_length = torch.tensor([waveform.shape[-1]]).int()
        
        tokens, token_lengths = self._tokenizer(transcript)
        
        return waveform, waveform_length, transcript, tokens, token_lengths
    
    def decode(self, tokens, lengths):
        result = []
        for tokens_, length in zip(tokens, lengths):
            text = "".join([
                self._tokenizer.tokens[token]
                for token in tokens_[:length]
            ])
            result.append(text)
        return result
                

In [18]:
from typing import Tuple, Dict, Optional, List, Union
from itertools import islice

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence


@dataclass
class Batch:
    waveform: torch.Tensor
    waveform_length: torch.Tensor
    transcript: List[str]
    tokens: torch.Tensor
    token_lengths: torch.Tensor
    durations: Optional[torch.Tensor] = None
        
    def to(self, device: torch.device) -> 'Batch':
        self.waveform = self.waveform.to(device)
        self.tokens = self.tokens.to(device)

        return self
    def _dur(self):
        return self.waveform_length // (MelSpectrogramConfig().hop_length)



class LJSpeechCollator:

    def __call__(self, instances: List[Tuple]) -> Dict:
        waveform, waveform_length, transcript, tokens, token_lengths = list(
            zip(*instances)
        )

        waveform = pad_sequence([
            waveform_[0] for waveform_ in waveform
        ]).transpose(0, 1)
        waveform_length = torch.cat(waveform_length)

        tokens = pad_sequence([
            tokens_[0] for tokens_ in tokens
        ]).transpose(0, 1)
        token_lengths = torch.cat(token_lengths)

        return Batch(waveform, waveform_length, transcript, tokens, token_lengths)

In [19]:
dataloader = DataLoader(LJSpeechDataset('.'), batch_size=3, collate_fn=LJSpeechCollator())

---

# Vocoder

In [20]:
!git clone https://github.com/NVIDIA/waveglow.git
!pip install googledrivedownloader

Cloning into 'waveglow'...
remote: Enumerating objects: 190, done.
remote: Total 190 (delta 0), reused 0 (delta 0), pack-reused 190
Receiving objects: 100% (190/190), 435.41 KiB | 10.88 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [21]:
from google_drive_downloader import GoogleDriveDownloader as gdd

In [22]:
gdd.download_file_from_google_drive(
    file_id='1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF',
    dest_path='./waveglow_256channels_universal_v5.pt'
)

In [23]:
import warnings
import sys
sys.path.append('waveglow/')

warnings.filterwarnings('ignore')


class Vocoder(nn.Module):

    def __init__(self):
        super(Vocoder, self).__init__()

        model = torch.load('waveglow_256channels_universal_v5.pt', map_location='cpu')[
            'model']
        self.net = model.remove_weightnorm(model)

    @torch.no_grad()
    def inference(self, spect: torch.Tensor):
        spect = self.net.upsample(spect)

        # trim the conv artifacts
        time_cutoff = self.net.upsample.kernel_size[0] - \
            self.net.upsample.stride[0]
        spect = spect[:, :, :-time_cutoff]

        spect = spect.unfold(2, self.net.n_group, self.net.n_group) \
            .permute(0, 2, 1, 3) \
            .contiguous() \
            .flatten(start_dim=2) \
            .transpose(-1, -2)

        # generate prior
        audio = torch.randn(spect.size(0), self.net.n_remaining_channels, spect.size(-1)) \
            .to(spect.device)

        for k in reversed(range(self.net.n_flows)):
            n_half = int(audio.size(1) / 2)
            audio_0 = audio[:, :n_half, :]
            audio_1 = audio[:, n_half:, :]

            output = self.net.WN[k]((audio_0, spect))

            s = output[:, n_half:, :]
            b = output[:, :n_half, :]
            audio_1 = (audio_1 - b) / torch.exp(s)
            audio = torch.cat([audio_0, audio_1], 1)

            audio = self.net.convinv[k](audio, reverse=True)

            if k % self.net.n_early_every == 0 and k > 0:
                z = torch.randn(
                    spect.size(0), self.net.n_early_size, spect.size(2),
                    device=spect.device
                )
                audio = torch.cat((z, audio), 1)

        audio = audio.permute(0, 2, 1) \
            .contiguous() \
            .view(audio.size(0), -1)

        return audio

In [24]:
vocoder = Vocoder().to(DEVICE).eval()

---

# Grapheme Aligner

In [25]:
@dataclass
class Point:
    token_index: int
    time_index: int
    score: float


@dataclass
class Segment:
    label: str
    start: int
    end: int
    score: float

    def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start:5d}, {self.end:5d})"

    @property
    def length(self):
        return self.end - self.start


class GraphemeAligner(nn.Module):

    def __init__(self):
        super().__init__()

        self._wav2vec2 = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H.get_model()
        self._labels = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H.get_labels()
        self._char2index = {c: i for i, c in enumerate(self._labels)}
        self._unk_index = self._char2index['<unk>']
        self._resampler = torchaudio.transforms.Resample(
            orig_freq=MelSpectrogramConfig.sr, new_freq=16_000
        )

    def _decode_text(self, text):
        text = text.replace(' ', '|').upper()
        return torch.tensor([
            self._char2index.get(char, self._unk_index)
            for char in text
        ]).long()

    @torch.no_grad()
    def forward(
        self,
        wavs: torch.Tensor,
        wav_lengths: torch.Tensor,
        texts: Union[str, List[str]]
    ):
        if isinstance(texts, str):
            texts = [texts]
        batch_size = wavs.shape[0]

        durations = []
        for index in range(batch_size):
            current_wav = wavs[index, :wav_lengths[index]].unsqueeze(dim=0)
            current_wav = self._resampler(current_wav)
            emission, _ = self._wav2vec2(current_wav)
            emission = emission.log_softmax(dim=-1).squeeze(dim=0).cpu()

            tokens = self._decode_text(texts[index])

            trellis = self._get_trellis(emission, tokens)
            path = self._backtrack(trellis, emission, tokens)
            segments = self._merge_repeats(texts[index], path)

            num_frames = emission.shape[0]
            relative_durations = torch.tensor([
                segment.length / num_frames for segment in segments
            ])

            durations.append(relative_durations)
            
        durations = pad_sequence(durations).transpose(0, 1)
        return durations

    def _get_trellis(self, emission, tokens, blank_id=0):
        num_frame = emission.size(0)
        num_tokens = len(tokens)

        # Trellis has extra dimension for both time axis and tokens.
        # The extra dim for tokens represents <SoS> (start-of-sentence)
        # The extra dim for time axis is for simplification of the code.
        trellis = torch.full((num_frame + 1, num_tokens + 1), -float('inf'))
        trellis[:, 0] = 0
        for t in range(num_frame):
            trellis[t + 1, 1:] = torch.maximum(
                # Score for staying at the same token
                trellis[t, 1:] + emission[t, blank_id],

                # Score for changing to the next token
                trellis[t, :-1] + emission[t, tokens],
            )
        return trellis

    def _backtrack(self, trellis, emission, tokens, blank_id=0):
        # Note:
        # j and t are indices for trellis, which has extra dimensions
        # for time and tokens at the beginning.
        # When refering to time frame index `T` in trellis,
        # the corresponding index in emission is `T-1`.
        # Similarly, when refering to token index `J` in trellis,
        # the corresponding index in transcript is `J-1`.
        j = trellis.size(1) - 1
        t_start = torch.argmax(trellis[:, j]).item()

        path = []
        for t in range(t_start, 0, -1):
            # 1. Figure out if the current position was stay or change
            # Note (again):
            # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
            # Score for token staying the same from time frame J-1 to T.
            stayed = trellis[t - 1, j] + emission[t - 1, blank_id]
            # Score for token changing from C-1 at T-1 to J at T.
            changed = trellis[t - 1, j - 1] + emission[t - 1, tokens[j - 1]]

            # 2. Store the path with frame-wise probability.
            prob = emission[t - 1, tokens[j - 1]
                            if changed > stayed else 0].exp().item()
            # Return token index and time index in non-trellis coordinate.
            path.append(Point(j - 1, t - 1, prob))

            # 3. Update the token
            if changed > stayed:
                j -= 1
                if j == 0:
                    break

        else:
            raise ValueError('Failed to align')

        return path[::-1]

    def _merge_repeats(self, text, path):
        i1, i2 = 0, 0
        segments = []
        while i1 < len(path):
            while i2 < len(path) and path[i1].token_index == path[i2].token_index:
                i2 += 1
            score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
            segments.append(
                Segment(
                    text[path[i1].token_index],
                    path[i1].time_index,
                    path[i2 - 1].time_index + 1,
                    score
                )
            )
            i1 = i2

        return segments

    @staticmethod
    def plot_trellis_with_path(trellis, path):
        # to plot trellis with path, we take advantage of 'nan' value
        trellis_with_path = trellis.clone()
        for i, p in enumerate(path):
            trellis_with_path[p.time_index, p.token_index] = float('nan')
        plt.imshow(trellis_with_path[1:, 1:].T, origin='lower')

In [26]:

aligner = GraphemeAligner().to(DEVICE)

Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth


  0%|          | 0.00/360M [00:00<?, ?B/s]

# Функции для тренировки 

## Логгирование аудио

Вроде можно как-то логгировать аудио прямо из тензора, но почему-то оно не работает.

Костыль: сохраню тензор в файлик, залоггирую файлик, удалю файлик.

In [27]:
import os
def add_audio_to_wandb(wav, transcript='', train=True):
    if train:
        key = 'train'
    else:
        key = 'val'

    path = 'audio_log_file.wav'
    with open(path, "wb") as f:
        f.write(wav.data)
    wandb.log(
          {key + '_audio': wandb.Audio(path, sample_rate=22050),
           key + '_transcript' : wandb.Html(transcript)})
    os.remove(path)

## Функция обучения

In [28]:
from tqdm import tqdm
def train(model,
          featurizer,
          aligner,
          loss_func,
          n_epochs,
          opt,
          scheduler,
          train_loader,
          val_loader=None,
          vocoder=None,
          num_model=0,
          logging=False
          ):
    
    for epoch in range(n_epochs):
        train_loss, train_mel_loss, train_dur_loss = train_epoch(model,
                    featurizer,
                    aligner,
                    loss_func,
                    opt,
                    scheduler,
                    train_loader,
                    vocoder,
                    num_model,
                    logging)
        print('\nepoch ' + str(epoch) + '/' + str(n_epochs) + ' train loss = ' + str(train_loss) + ' mel loss = ' + str(train_mel_loss) + ' dur loss = ' + str(train_dur_loss))
        torch.save(model.state_dict(), 'model_' + str(num_model) +'.pt')
        torch.save(model.state_dict(), '/content/drive/MyDrive/tts_models/model_' + str(num_model) +'.pt')
        if logging:

            wandb.log({
              'train_loss_epoch' : train_loss,
              'train_mel_loss_epoch' : train_mel_loss,
              'train_dur_loss_epoch' : train_dur_loss
            })
        
        
        
def train_epoch(model,
                    featurizer,
                    aligner,
                    loss_func,
                    opt,
                    scheduler,
                    train_loader,
                    vocoder=None,
                    num_model=0,
                    logging=False):
    model.train()
    model = model.to(DEVICE)
    loss_sum, mel_loss_sum, dur_loss_sum = 0, 0, 0
    n_corrupted_batches = 0
    for batch in tqdm(train_loader, total=len(train_loader)):
        #batch = batch.to(DEVICE)
        mel_real = featurizer(batch.waveform.to(DEVICE))
        with torch.no_grad():
            
            batch.durations = aligner(batch.waveform.to(DEVICE), batch.waveform_length.to(DEVICE), batch.transcript).to(DEVICE)
        #print('during', batch.durations.shape, batch.tokens.shape)
        if batch.durations.shape[-1] !=  batch.tokens.shape[-1]:
            print('corrupted batch')
            n_corrupted_batches += 1
            continue

        mel_dur = batch._dur().to(DEVICE)
        batch.real_durations = torch.mul(batch.durations, mel_dur.unsqueeze(1))

        opt.zero_grad()
        #print(batch.tokens.shape, batch.real_durations.shape)
        mel_pred, dur_pred = model(batch.tokens.to(DEVICE), batch.real_durations.to(DEVICE))
       

        mel_loss, dur_loss = loss_func(mel_real, mel_pred.transpose(-2,-1), batch.real_durations, dur_pred)

        #Вроде надо просто взять сумму. или взвешенную сумму?
        #TODO понять
        loss = mel_loss + dur_loss
        mel_loss_sum += mel_loss.item()
        dur_loss_sum += dur_loss.item()
        loss_sum += loss.item()
        if logging:

            wandb.log({
              'train_loss_step' : loss.item(),
              'train_mel_loss_step' : mel_loss.item(),
              'train_dur_loss_step' : dur_loss.item()
            })

        loss.backward()
        opt.step()
    if vocoder is not None and mel_pred is not None and logging:
        reconstructed_wav = vocoder.inference(mel_pred[0].transpose(-2,-1).unsqueeze(0)).cpu()
        wav = display.Audio(reconstructed_wav, rate=22050)
        add_audio_to_wandb(wav, transcript=batch.transcript[0], train=True)
    len_ = len(train_loader) - n_corrupted_batches
    return loss_sum /  len_, mel_loss_sum / len_, dur_loss_sum / len_



     

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
#!g1.1
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fast_speech = FastSpeechModel(#Это для энкодера
                 vocab_size=10000, #В статье вроде этого нет. ИЗначально брала 1000, умные люди сказали взять 10000
                 n_layers_enc=6,
                 embed_dim=384, 
                 num_heads_enc=2,
                 hidden_dim_enc=1536, 
                 kernel_size_1_enc=3, 
                 kernel_size_2_enc=3, 
                 dropout1_enc=0.1,
                 dropout2_enc=0.1,
                 #Это для length_regulator
                 #embed_dim, Не надо, так как нужно то же самое, что и в энкодере
                 hidden_dim_lr=384,
                 kernel_size_1_lr=3,
                 kernel_size_2_lr=3,
                 dropout1_lr=0.1,
                 dropout2_lr=0.1,
                 #Это для декодера
                 n_layers_dec=6,
                 out_dim=80,
                 #embed_dim, Не надо, так как нужно то же самое, что и в энкодере
                 num_heads_dec=2,
                 hidden_dim_dec=1536, 
                 kernel_size_1_dec=3, 
                 kernel_size_2_dec=3, 
                 dropout1_dec=0.1,
                 dropout2_dec=0.1)
loss_func =  LossFunctionBlock()

In [41]:
#num_resume=9
#fast_speech.load_state_dict(torch.load('/content/drive/MyDrive/tts_models/model_'+str(num_resume)+'.pt'))
opt = torch.optim.Adam(fast_speech.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [31]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 12.8 MB/s 
     |████████████████████████████████| 97 kB 9.3 MB/s 
     |████████████████████████████████| 140 kB 70.6 MB/s 
     |████████████████████████████████| 180 kB 65.5 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=e7da3b7dcbe7609ca6654e9d641fb002d62da7764e7fe2b787ac39ccc2a20d77
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=98f6f189c94070c17f8342283a8536c9c4eec56444ecd26eaa667983b2f1aed5
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


In [58]:
import wandb
wandb.login()
wandb.init(project="full_data_colab_testing")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kogana00 (use `wandb login --relogin` to force relogin)


In [ ]:
#У этой ячейки нет output, так как эта реализует весь процесс обучения, который быд на самом деле разбит
n_epochs = 84
num_model = 14 #Это аргумент для сохранения модели, чтобы сохранялось в файл model_*num_model*.pt
vocoder = Vocoder().to(DEVICE).eval()
train(model=fast_speech.to(DEVICE),
          featurizer=featurizer.to(DEVICE),
          aligner=aligner,
          loss_func=loss_func,
          n_epochs=n_epochs,
          opt=opt,
          scheduler=None,
          train_loader=dataloader,
          val_loader = None,
          vocoder = vocoder,
          num_model=num_model,
          logging=True
          )

In [60]:
#А это ячейка с примером output
'''vocoder = Vocoder().to(DEVICE).eval()
train(model=fast_speech.to(DEVICE),
          featurizer=featurizer.to(DEVICE),
          aligner=aligner,
          loss_func=loss_func,
          n_epochs=10,
          opt=opt,
          scheduler=None,
          train_loader=dataloader,
          val_loader = None,
          vocoder = vocoder,
          num_model=10,
          logging=True
          )'''

  0%|          | 3/4367 [00:01<24:26,  2.98it/s]

corrupted batch


  0%|          | 17/4367 [00:06<22:10,  3.27it/s]

corrupted batch


  1%|          | 32/4367 [00:11<23:29,  3.08it/s]

...

corrupted batch


 89%|████████▉ | 3879/4367 [22:53<02:39,  3.07it/s]

corrupted batch


100%|██████████| 4367/4367 [25:50<00:00,  2.82it/s]



epoch 9/10 train loss = 2.868512814718682 mel loss = 1.0286795067100805 dur loss = 1.8398333073121844


# Тест



In [32]:
test_sent = ['a defibrillator is a device that gives a high energy electric shock to the heart of someone who is in cardiac arrest',
'Massachusetts Institute of Technology may be best known for its math, science and engineering education',
'Wasserstein distance or Kantorovich Rubinstein metric is a distance function defined between probability distributions on a given metric space']
ds = LJSpeechDataset('.')
test_toks_lengths = [ds._tokenizer(sent) for sent in test_sent]

In [33]:
def test(model,
         test_toks_lengths, #Массив из tuple, содержит токены и предсказанную токенизатором длину
         vocoder):
    model.eval()
    model = model.to(DEVICE)
    
    mel_preds = []

    with torch.no_grad():
        for toks, lens in test_toks_lengths:
        
            mel_pred, dur_pred = model(toks.to(DEVICE))
            mel_preds.append(mel_pred)
    return mel_preds
    
      
      

         

In [45]:
#Загрузка финальной модели, если не было обучения
model_num=13
fast_speech.load_state_dict(torch.load('/content/drive/MyDrive/tts_models/model_'+str(model_num)+'.pt'))


<All keys matched successfully>

In [35]:
mel_preds = test(fast_speech, test_toks_lengths, vocoder)

In [37]:
reconstructed_wav = vocoder.inference(mel_preds[0].transpose(-2,-1)).cpu()
torchaudio.save('test0_model_'+str(model_num)+'.wav', reconstructed_wav, 22050)
display.display(display.Audio(reconstructed_wav,rate=22050))

In [38]:
reconstructed_wav = vocoder.inference(mel_preds[1].transpose(-2,-1)).cpu()
torchaudio.save('test1_model_'+str(model_num)+'.wav', reconstructed_wav, 22050)
display.display(display.Audio(reconstructed_wav,rate=22050))

In [39]:
reconstructed_wav = vocoder.inference(mel_preds[2].transpose(-2,-1)).cpu()
torchaudio.save('test2_model_'+str(model_num)+'.wav', reconstructed_wav, 22050)
display.display(display.Audio(reconstructed_wav,rate=22050))